# Image Processing in Python

In this notebook, we will go through some basic image processing in Python!  

Then we'll take a look at a machine learning application called Style Transfer to do some really wild modern image processing.

### 1.1 Basic Image Processing and Manipulation
First, we need to import some packages that provide us with tools for manipulating images. 

We will also need to import a nice image to play with.

In [ ]:
!unzip -e HOTDemo-master.zip

In [ ]:
# Import useful packages for image manipulation and plotting
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread, imsave

In [ ]:
# Import an image to play with
from skimage import data
cat = data.chelsea()

#### 1.2 Plotting the image and converting to grayscale
Let's take a look at the image, in both color and grayscale.

In [ ]:
from skimage.color import rgb2grey

# Set up a figure to plot images in
f = plt.figure(1, figsize=(15,5))

# Convert the image to grayscale
gray_cat = (rgb2grey(cat)*255).astype('uint8')

# Display the cat in color!
plt.subplot(121)
plt.imshow(cat)
plt.title('Chelsea : Size = ' + str(cat.shape))
plt.axis('off')

# Display the cat in gray!
plt.subplot(122)
plt.imshow(gray_cat,cmap='gray')
plt.title('Gray Chelsea : Size = ' + str(gray_cat.shape))
plt.axis('off')
plt.show()

Notice how the size of each image is the same, 300 x 451 pixels, but the color image has three separate color channels!

#### 1.3 Plotting out the R, G, B channels separately
Each pixel of a digital image is defined by how much of the colors red, green, and blue it contains, which are called "channels". 

For example, a white pixel in uint8 encoding is [255, 255, 255] (full RGB intensity), while a black pixel is [0, 0, 0] (zero RGB intensity), and a red pixel is [255, 0, 0] (full R intensity, zero GB intensity). 

We can split an image into its 3 channels to get an idea of how much of each color is present in each pixel.

In [ ]:
f = plt.figure(1, figsize=(15,5))

# Pull out the number of color channels and give them names
num_c = cat.shape[-1]
colors = ['Red','Green','Blue']

# Plot each color channel as a separate image
for ii, clr in enumerate(colors):
    plt.subplot(1, num_c, ii+1)
    plt.imshow(cat[:, :, ii], cmap=clr+'s_r')
    plt.title('Chelsea : ' + clr + ' channel')
    plt.axis('off')
plt.show()

#### 1.4 Cropping & Flipping
Cropping and flipping is easy - we just have to change the bounds of the pixels to crop, or the order of the rows or columns of the image to flip it. 

In [ ]:
# Crop the image to be square
#  - Take a look at the image size from Section 1.2 to remind yourself how many 
#  - pixels the image has in each dimension
# 
#  - Try changing the indices below to center the cat's face within the crop
cat_sq = gray_cat[:, 0:300]

# Flip the image horizontally
cat_sq_flipH = cat_sq[:, ::-1]

# Flip the image vertically
cat_sq_flipV = # FILL CODE HERE 

In [ ]:
# Plot the square-cropped and both flipped versions of the cat image
f = plt.figure(1, figsize=(15,5))

plt.subplot(131)
plt.imshow(cat_sq,cmap='gray')
plt.title('Square Cat\nSize = ' + str(cat_sq.shape))
plt.axis('off')

plt.subplot(132)
plt.imshow(cat_sq_flipH,cmap='gray')
plt.title('Horizontal Flip\nSize = ' + str(cat_sq_flipH.shape))
plt.axis('off')

plt.subplot(133)
plt.imshow(cat_sq_flipV,cmap='gray')
plt.title('Vertical Flip\nSize = ' + str(cat_sq_flipV.shape))
plt.axis('off')

plt.show()

#### 1.5 Median filtering 
A common problem in image processing is removing "noise" - some sort of corruption which makes the image less clean. A popular solution is to implement a median filter, which replaces each pixel with the median of the pixels around it.

First we'll corrupt our image with random noise. Then, we'll try to recover the original image by applying a median filter.

In [ ]:
# Import packages for adding noise, measuring image corruption, and filtering
from skimage.util import random_noise
from skimage.measure import compare_psnr, compare_ssim
from skimage.filters import median

In [ ]:
# Add random noise to the cat picture
#  - Here we add 'Salt and Pepper' noise
#  - Take a look at some of the other types of image noise with the link below
#  - https://scikit-image.org/docs/dev/api/skimage.util.html#skimage.util.random_noise
salty_cat_sq = (random_noise(cat_sq, mode='s&p') * 255).astype('uint8')

# Apply a median filter to reduce the noise
less_salty = median(salty_cat_sq)

# Plot the noisy and filtered images
f = plt.figure(1, figsize=(10,5))

plt.subplot(121)
plt.imshow(salty_cat_sq,cmap='gray')
plt.title('Salty Cat\nPSNR = ' + str(np.around(compare_psnr(cat_sq, salty_cat_sq),4)))
plt.axis('off')

plt.subplot(122)
plt.imshow(less_salty,cmap='gray')
plt.title('Less Salty Cat\nPSNR = ' + str(np.around(compare_psnr(cat_sq, less_salty),4)))
plt.axis('off')

plt.show()

In the 'Salty Cat' and 'Less Salty Cat' images above, PSNR means Peak Signal-to-Noise Ratio and is a measure of image corruption. 

>*Judging from the two images above - is a high PSNR good or bad?*

#### 1.6 Implementing different types of filters with Convolution
In the context of image processing, "convolution" is the mathematical operation that performs filtering. Convolution defines each pixel in the new image as a weighted sum of the original pixels in a square region around that pixel. The weights and size of the region define a convolution filter, commonly called a "kernel". Implementing convolution means applying this kernel to every pixel in the original image to create the new image.

For example, we can create a simple blurring effect by defining our kernel to be the average of all of the pixels in a 3 x 3 square. 

There are a ton of interesting kernels out there that do things like reduce image noise or highlight eye-catching image features. Let's define some other cool kernels and see how convolving our image with them can introduce some pretty cool effects!

In [ ]:
# Import a package to perform 2D convolution
from scipy.signal import convolve2d

In [ ]:
# Define some basic filters and apply them to the cat image

# Basic blur filter
fblur = 1/9 * np.array([[1, 1, 1],
                        [1, 1, 1],
                        [1, 1, 1]])
cat_sq_blur = convolve2d(cat_sq, fblur, mode='same', boundary='symm')

# Vertical edge detection filter
fvedge = np.array([[-1,0,1],
                   [-2,0,2],
                   [-1,0,1]])
cat_sq_ve = convolve2d(cat_sq, fvedge, mode='same', boundary='symm')

# Embossing filter
femboss = np.array([[-2,1,0],
                    [-1,1,1],
                    [0,1,2]])
cat_sq_emboss = convolve2d(cat_sq, femboss, mode='same', boundary='symm')

In [ ]:
# Plot the filtered images
f = plt.figure(1, figsize=(15,5))

plt.subplot(131)
plt.imshow(cat_sq_blur, cmap='gray')
plt.title('Blurring')
plt.axis('off')

plt.subplot(132)
plt.imshow(cat_sq_ve, cmap='gray')
plt.title('Vertical Edge Detection')
plt.axis('off')

plt.subplot(133)
plt.imshow(cat_sq_emboss,cmap='gray')
plt.title('Embossing')
plt.axis('off')

plt.show()

>When convolved with an identity filter, an all pixels values in the image stay the same. *What would the identity filter be for a kernel size of 3x3?*

>*What filter would shift all pixels in an image to right by one? (Hint : it looks very similar to the identity filter!)*

#### 1.7 Effects of changing the kernel size
Of course, we're not limited to a 3x3 kernel. Going back to blurring an image, a larger blurring kernel means more blurring!

In [ ]:
# 3x3 blurring kernel we already defined in Section 1.6
k3 = 1/9 * np.array([[1, 1, 1],
                     [1, 1, 1],
                     [1, 1, 1]])

# Change this to blur with a 5x5 kernel!
# FILL CODE HERE
k5 = 1/????? * np.array([[0,0,0,0,0],
                     [0,0,0,0,0],
                     [0,0,1,0,0],
                     [0,0,0,0,0],
                     [0,0,0,0,0]])


# Change this to blur with a 7x7 kernel!
# FILL CODE HERE
k7 = 1/????? * np.array([[0,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0],
                     [0,0,0,1,0,0,0],
                     [0,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0]])


cat_sq_3 = convolve2d(cat_sq, k3, mode='same', boundary='symm').astype('uint8')
cat_sq_5 = convolve2d(cat_sq, k5, mode='same', boundary='symm').astype('uint8')
cat_sq_7 = convolve2d(cat_sq, k7, mode='same', boundary='symm').astype('uint8')

In [ ]:
# Plot three levels of blurring
f = plt.figure(1, figsize=(15,5))

plt.subplot(131)
plt.imshow(cat_sq_3, cmap='gray')
plt.title('3x3 Blurring\nSSIM = '+str(np.around(compare_ssim(cat_sq,cat_sq_3),4)))
plt.axis('off')

plt.subplot(132)
plt.imshow(cat_sq_5, cmap='gray')
plt.title('5x5 Blurring\nSSIM = '+str(np.around(compare_ssim(cat_sq,cat_sq_5),4)))
plt.axis('off')

plt.subplot(133)
plt.imshow(cat_sq_5, cmap='gray')
plt.title('7x7 Blurring\nSSIM = '+str(np.around(compare_ssim(cat_sq,cat_sq_7),4)))
plt.axis('off')

plt.show()

SSIM (Structural Similarity Index) above is another measure of image corruption that ranges from zero (entirely dissimilar images) to one (exactly the same image).

>*How do the SSIM scores above compare to your opinion on how similar these images look to the original?*

#### 1.8 Edge Detection
A common problem for modern autonomous systems (like self-driving cars or robots) is object recognition. If we want to figure out whether an image contains a certain object, the first step is to find the edges of objects in the image since machines, like humans, rely heavily on edges to understand what they see. 

More specifically, we'd like to create a new image where large values (white or black pixels) correspond to pixels where we think there's an edge. We've already seen a kernel that can do this for vertical edges in Section 1.6. 

An edge in an image is usually defined by a sharp difference between adjacent pixels. If a kernel substracts values on one side of a pixel from values on the other, we have a simple edge detection kernel that's called a Sobel kernel. This kernel will output values near zero when adjacent pixels are similar, and large values when adjacent pixels are substantially different.

In [ ]:
# Define two types of edge detection filters

# Vertical edge detection filter we already defined in Section 1.6
sobel_v = np.array([[-1,0,1],
                    [-2,0,2],
                    [-1,0,1]])

# Fill in the values below to define a horizontal edge detection filter
# - Hint : there's a simple relationship between horizontal and vertical edges!
sobel_h = # FILL CODE HERE

In [ ]:
# Filter the cat image with vertical and horizontal edge detection filters
cat_sq_v = convolve2d(cat_sq, sobel_v, mode='same', boundary='symm')
cat_sq_h = convolve2d(cat_sq, sobel_h, mode='same', boundary='symm')

# Compute the combined edge magnitude of both edge directions
cat_sq_all = np.sqrt( cat_sq_v**2 + cat_sq_h**2 ) / np.sqrt(2)

In [ ]:
# Plot the vertical, horizontal, and combined edge-detected images
f = plt.figure(1, figsize=(15,5))

plt.subplot(131)
plt.imshow(cat_sq_v, cmap='gray')
plt.title('Vertical Edges')
plt.axis('off')

plt.subplot(132)
plt.imshow(cat_sq_h, cmap='gray')
plt.title('Horizontal Edges')
plt.axis('off')

plt.subplot(133)
plt.imshow(cat_sq_all, cmap='gray')
plt.title('Combined Edges')
plt.axis('off')

plt.show()

>*What differences in the vertical and horizontal edge-detected images sticks out the most?*

>*Do you notice anything weird happening in the combined edge image?*

### 2 Style Transfer

In [ ]:
import os
import sys
sys.path.insert(0, '/content/HOTDemo-master/HOT_Demo') #imports style transfer
workingdir = '{}'.format(os.getcwd()) + '/HOTDemo-master/HOT_Demo/'
import style_transfer

In [ ]:
from style_transfer import magic_box
import os

content_image = 'shenyang.jpg'
style_image = 'pencil.jpg'
out_name = 'Pencil-Shenyang.jpg'
cff = True #true if the content image is already in the content folder
sff = True #true if the style image is already in the 21styles folder 

new_img = style_transfer.magic_box(content_image,style_image,out_name,workingdir,cff,sff)

plt.figure()
plt.imshow(new_img)
dp = ([pos for pos, char in enumerate(out_name) if char == '.'])
plt.title(str(out_name[:dp[0]])+': Size = '+str(new_img.shape))
plt.axis('off')

Following is an example that shows how you can use content (and even style!) images of your own

In [ ]:
new_content = cat_sq
#new_content = imread('img.jpg')

content_image = new_content
style_image = 'mosaic.jpg'
out_name = 'Mosaic-Chelsea.jpg'
cff = False #true if the style image is already in the content folder
sff = True #true if the style image is already in the 21styles folder

new_img = magic_box(content_image,style_image,out_name,workingdir,cff,sff)

plt.figure()
plt.imshow(new_img)
dp = ([pos for pos, char in enumerate(out_name) if char == '.'])
plt.title(str(out_name[:dp[0]])+': Size = '+str(new_img.shape))
plt.axis('off')